In [ ]:
import pymongo
import re
import pprint
import json
import jieba
from collections import Counter
import time
import math
import logging
from gensim.models import word2vec
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

In [ ]:
with open ("../data/diendata.json") as f:
    ipeendata=json.load(f)    

In [ ]:
ipeendata[0]

In [ ]:
n=0
for dien in ipeendata:
    if dien['Ncomment']!=0:
        
        try:
            comments=json.load(open("../data/diencomments/"+str(dien['id'])+".json"))
            n+=1
            if n%1000==0:
#                 print("已讀取{}筆資料".format(n))
        except Exception as e:
            print(e)
            comments=[]
    else:
        comments=[]
    dien["comments"]=comments
print("讀取完畢")

In [ ]:
ipeendata=[dien for dien in ipeendata if dien['Ncomment'] >0 and dien['address']!="" and dien['status']=='正常營業' and dien['comments']!=[]]
print(len(ipeendata))
# ipeendata[50]

In [ ]:
jieba.set_dictionary('D:/dict/jieba_dict.txt.big')

In [ ]:
#停止慈字典
stopwordset = set()
with open('D:/dict/stopwords.txt', 'r') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
        
#自製餐廳相關字字典pinlist
with open("D:/dict/FoodMoodDict.csv") as f:
    mydict=f.read()
#linewords pindict每個字的評分dict
linewords=[{myword:float(line.split(",")[0]) for myword in line.split(",")[1:]}\
           for line in mydict.split("\n") if line.split(",")[0]!=""]
pindict={}
for word in linewords:
    pindict.update(word)
pinlist=list(pindict.keys())

In [ ]:
ipeendata[5]

In [ ]:
#各種資料list
dienlist = [dien["name"] for dien in ipeendata]
bigstylelist = [dien["bigstyle"] for dien in ipeendata]
smallstylelist = [dien["smallstyle"] for dien in ipeendata]



#將以上字典都加入斷詞行列
for i in dienlist:
    jieba.add_word(i)
for i in bigstylelist:
    jieba.add_word(i)

for i in smallstylelist:
    jieba.add_word(i)
for i in pinlist:
    if i !="":
        jieba.add_word(i)
        
latlist = [dien['lat'] for dien in ipeendata]
lnglist = [dien['lng'] for dien in ipeendata]
Ncommentlist = [i["Ncomment"] for i in ipeendata]

In [ ]:
diencommentlist=[[j["content"] for j in i["comments"]] for i in ipeendata]
diencommentlist[5]

In [ ]:
dienmessagelist=[[''.join(j["message"]) for j in i["comments"] if j["message"]!=None and j["message"]!='null'] for i in ipeendata]
dienmessagelist[10]

In [ ]:
#回覆數量
Nmessagelist=[len(dienmessage) for dienmessage in dienmessagelist]

In [ ]:
#全部回覆+全部評論
dienallpinlist=[diencomment+dienmessage for diencomment,dienmessage in zip(diencommentlist,dienmessagelist)]
# dienallpinlist[10]

In [ ]:
#把所有回覆合併
messagecombinlist=[" ".join(["".join(meg) for meg in megs if meg!=None and meg!='null'and meg!='']) for megs in dienmessagelist]
# messagecombinlist[10]

In [ ]:
#把所有評論合併
commentcombinelist = [" ".join([j["content"] for j in i["comments"]]) for i in ipeendata]
# commentcombinelist[10]

In [ ]:
#合併評論+回覆
pincombinelist=[comment+" "+message for comment,message in zip(commentcombinelist,messagecombinlist)]
# pincombinelist[10]

In [ ]:
#清一下記憶體
import gc
gc.collect()

In [ ]:
st=time.time()
#取得各店家的斷完詞評論pincutlist
pincutlist = []
cutcount=0
for pincombine in pincombinelist:
    if pincombine!="":
        cutcount+=1
        if cutcount%200==0:
            print("已切"+str(cutcount)+"篇文章的詞")
        words = jieba.cut(pincombine, cut_all=False)
        pincut=" ".join([word for word in words if word not in stopwordset and '\u4e00' <= word and word <= '\u9fff'])
        pincutlist.append(pincut)
    else:
        pincutlist.append("")
et=time.time()
print(et-st)

In [ ]:
#清一下記憶體
import gc
gc.collect()

In [ ]:
from myapi import TextMining
TM=TextMining.TextMining(pincutlist)
wordCountlist=TM.getWordCountList()
TF_IDFlist=TM.getTFIDF()

In [ ]:
# #所有店家的用詞加總前三百wordCountlist300
# wordCountlist=[Counter(pincut.split(" ")).most_common(300) for pincut in pincutlist]

# st=time.time()
# #TF計算 (TF=這個字出現的次數/這篇文章的總字數)
# #(sum(dict(wordCount).values())=>一篇所有字的字數  wordkv[1]>>該文字字數  wordkv[0]>>該文字)
# #TFList每篇文章的詞 的詞頻
# # TFList=[[((wordkv[0],wordkv[1]/sum(dict(wordCount).values()))) for wordkv in wordCount] for wordCount in wordCountlist]
# n=0
# TFList=[]
# for wordCount in wordCountlist:
#     n+=1
#     if n%1000==0:
#         print(n)
#     TF={}
#     allwords=sum(dict(wordCount).values())
#     for wordkv in wordCount:
#         TF[wordkv[0]]=(wordkv[1]/allwords)
#     TFList.append(TF)
    

# #全文章共用了幾個詞(set可以去除重複，可迭代)
# wordlist=[]
# for wordCount in wordCountlist:
#     for word in wordCount:
#         wordlist.append(word[0])
# wordlist=set(wordlist)

# #nDien所有的文章(店家)數
# ndien=len(dienlist)

# et=time.time()
# print(et-st)

In [ ]:
# a=set()

In [ ]:
# len(wordlist)

In [ ]:
#清一下記憶體
import gc
gc.collect()

In [ ]:
# #計算各字出現次數
# st=time.time()
# wordappear={}
# k=0
# for wordCount in wordCountlist:
#     k+=1
#     if k%3000==0:
#         print(k)
#     for word in dict(wordCount).keys():
#         if word in wordappear:
#             wordappear[word]+=1
#         else:
#             wordappear[word]=1
            
# et=time.time()
# print(et-st)

In [ ]:
# del wordappear[""]

In [ ]:
# # #wordappear計算每個字出現於 (幾篇) 文章
# st=time.time()
# # wordappear={}
# # appearcount=0
# # for word in wordlist:
# #     appearcount+=1
# #     if appearcount%1000==0:
# #         print("以計算"+str(appearcount)+"個文字的出現篇數")
# #     n=0
# #     for pincut in pincutlist:
# #         if word in pincut:
# #             n+=1
# #     wordappear[word]=n
# #IDF(逆向檔案頻率)=某詞  所有文章數/在幾篇文章出現 開log10 >>次方數為所
# IDFlist={word:math.log(ndien/wordappear[word],10) for word in wordappear}


    
# et=time.time()
# print(et-st)

In [ ]:
# len(IDFlist)

In [ ]:
# TFList[2]

In [ ]:
# #dienTF每家店的全字TF值 wordTF每個詞的TF   wordTF[0]詞 wordTF[1]詞的TF值  IDFlist[wordTF[0]]詞的IDF值
# # TF_IDFlist=[Counter({wordTF[0]:wordTF[1]*IDFlist[wordTF[0]]for wordTF in dienTF})for dienTF in TFList]
# # for name,TFIDF in zip(dienlist,TF_IDFlist):
# #     if name in TFIDF:
# #         del TFIDF[name]
# n=0
# TF_IDFlist=[]
# for dienTF in TFList:
#     n+=1
#     if n%3000==0:
#         print(n)
#     x=Counter()
#     for wordTF in dienTF:
#         x.update({wordTF:dienTF[wordTF]})
#     TF_IDFlist.append(x)

In [ ]:
for name,TFIDF in zip(dienlist,TF_IDFlist):
    if name in TFIDF:
        del TFIDF[name]

In [ ]:
gc.collect()

In [ ]:
# st=time.time()
# pinCountMaxper5list = []
# countN=0
# for dienallpin in dienallpinlist:
#     dienWordcount={}
#     for allpin in  dienallpin:
#         countN+=1
#         if countN%100==0:
#             print("已算完"+str(countN)+"篇情緒字")
#         for pinword in pinlist:
#             if len(re.findall(pinword,allpin))>0:
#                 if len(re.findall(pinword,allpin))>5:
#                     npinword=5
#                 else:
#                     npinword=len(re.findall(pinword,allpin))
#                 if pinword not in dienWordcount:
#                     dienWordcount[pinword]=0
#                 dienWordcount[pinword]+=npinword
#     pinCountMaxper5list.append(dienWordcount)
# ed=time.time()
# print(ed-st)

In [ ]:
st=time.time()
pinCountMaxper5list = []
countN=0
for dienallpin in dienallpinlist:
    dienWordcount={}
    for allpin in  dienallpin:
        countN+=1
        if countN%100==0:
            print("已算完"+str(countN)+"篇情緒字")
        for pinword in pinlist:
            nword=len(re.findall(pinword,allpin))
            if nword>0:
                if nword>5:
                    npinword=5
                else:
                    npinword=nword
                if pinword not in dienWordcount:
                    dienWordcount[pinword]=0
                dienWordcount[pinword]+=npinword
    pinCountMaxper5list.append(dienWordcount)
ed=time.time()
print(ed-st)

In [ ]:
Npartpin=[int(Nmessage/6)+Ncomment for Nmessage,Ncomment in zip(Nmessagelist,Ncommentlist)]


#大致計算每家店的分數dienscorelist
dienscorelist=[]
for pinCount,N in zip(pinCountMaxper5list,Npartpin):
    dienscore=0
    for pin in pinCount:
        dienscore+=pinCount[pin]*pindict[pin]/N
    dienscorelist.append(dienscore)

In [ ]:
synonyms=[{word:line.split(",")[1] for word in line.split(",")[1:]} for line in mydict.split("\n") if line.split(",")[0]!=""]

pinsynonymsdict={}
for synonym in synonyms:
    pinsynonymsdict.update(synonym)
del pinsynonymsdict[""]

In [ ]:
featurescores=[]

#將評價詞加總及評論次數做iterate
for pinCountMaxper5,N in zip(pinCountMaxper5list,Npartpin):
    dienfeaturescore={}
    
    # 評價詞出現次數/評論次數 做為分數，其中否字開頭給負分
    for pin in pinCountMaxper5:
        if pin !="":
            if pinsynonymsdict[pin][0]=="否":
                if pinsynonymsdict[pin][1:] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin][1:]]=0
                dienfeaturescore[pinsynonymsdict[pin][1:]]-=round(pinCountMaxper5[pin]/N,2)*1
            else:
                if pinsynonymsdict[pin] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin]]=0
                dienfeaturescore[pinsynonymsdict[pin]]+=round(pinCountMaxper5[pin]/N,2)
    if len(dienfeaturescore)==0:
        dienfeaturescore["無評"]=1
    featurescores.append(dienfeaturescore)

In [ ]:
BigAnalyzeTable=[]
for name,bigstyle,smallstyle,Ncomment,contentcut,wordCount,TFIDF,pinCountMax5,score,featurescore,introduction,lat,lng in zip(dienlist,bigstylelist,smallstylelist,Npartpin,pincutlist,wordCountlist,TF_IDFlist,pinCountMaxper5list,dienscorelist,featurescores,latlist,lnglist):
    dien={}
    dien['name']=name
    dien['bigstyle']=bigstyle
    dien['smallstyle']=smallstyle
    dien['Ncomment']=Ncomment
    dien['contentcut']=contentcut
    dien['wordCount']=wordCount
    dien['TF_IDF']=TFIDF
    dien['pinCountMaxper5']=pinCountMax5
    dien['score']=score
    dien['featurescores']=featurescore
    dien['tags']=[tag[0] for tag in Counter(TFIDF).most_common(5)]
    dien['lat']=lat
    dien['lng']=lng
    
    
    BigAnalyzeTable.append(dien)

In [ ]:
gc.collect()

In [ ]:
#算出每個評論的評論數N時的 最大值b 最小值g for正規化


#allpin=評論詞list
allpin=set([line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"])
standard={}

#standard為正規化所需用的值
for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
        get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
        get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
        standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

In [ ]:
#進行正規化的處理
for dien in BigAnalyzeTable:
    N=dien['Ncomment']
    for feature in dien['featurescores']:
        if feature !="無評":
            if N<6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(0)][1],2)
            elif N<11 and N>=6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(5)][1],2)
            elif N<16 and N>=11:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(10)][1],2)
            else:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(15)][1],2)

In [ ]:
allpin=[line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"]
standard={}

for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
#         get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
#         get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
#         standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

In [ ]:
pipin=[('食物美味','食物不好吃',"非常好吃",'非常不好吃'),('划算','價格較高昂',"非常便宜",'非常貴')
       ,('環境好','環境比較不好',"環境超棒",'環境很差'),('服務好','服務較差',"服務一級棒",'服務很差')
       ,('其他正面情緒','評論有些負面情緒詞',"網路評論正面情緒非常多",'網路評論負面情緒很多')]
moods=[mood[0] for mood in pipin]
for dien in BigAnalyzeTable:
    newtag=[]
    for feature in dien["featurescores"]:
        if feature in moods:
            if dien["featurescores"][feature]>1:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[2])
            elif dien["featurescores"][feature]>0.4:
                newtag.append(feature)
            elif dien["featurescores"][feature]<-0.5:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[3])
            elif dien["featurescores"][feature]<0.0:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[1])
        else:
            if dien["featurescores"][feature]>0.7:
                newtag.append(feature)
                    
                    
#用set去除重複標籤
    dien['tags']=list(set(dien['tags']+newtag))
    print(dien['name'])
#     print(newtag)
    dien['tags'].append(dien['bigstyle'])
    print(dien['tags'])

In [ ]:
for i in BigAnalyzeTable:
    if i['score']>30 and i['Ncomment']>10:
        print(i['name']+' '+str(i['score']))
#         print(i['tags'])
        print(i['featurescores'])
        print('\n')
gooddien=[i for i in BigAnalyzeTable if i['score']>30 and i['Ncomment']>10]

In [ ]:
with open('../data/bigtable.json','w') as f:
    json.dump(BigAnalyzeTable,f)
with open('../data/GoodDien.json','w') as f:
    json.dump(gooddien,f)   

In [ ]:
len(BigAnalyzeTable[3]['lng'])

# 試做word2vec

In [ ]:
from gensim.models import word2vec
import logging

In [ ]:
import json
with open('../data/bigtable.json') as f:
    BigAnalyzeTable=json.load(f)

In [ ]:
contentcutcombine=" ".join([i['contentcut'] for i in BigAnalyzeTable])
with open('../data/contentcutcombine.txt','w',encoding="utf-8") as w:
    w.write(contentcutcombine)

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.Text8Corpus('../data/contentcutcombine.txt')
model = word2vec.Word2Vec(sentences, size=250,window=6, min_count=4, workers=4,sg=1)

# Save our model.

model.save("../data/W2Vmodel/med250.model.bin")

In [ ]:
from gensim.models import word2vec
from gensim import models
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = models.Word2Vec.load('../data/W2Vmodel/med250.model.bin')

print("提供 3 種測試模式\n")
print("輸入一個詞，則去尋找前一百個該詞的相似詞")
print("輸入兩個詞，則去計算兩個詞的餘弦相似度")
print("輸入三個詞，進行類比推理")

while True:
    try:
        query = input()
        q_list = query.split()

        if len(q_list) == 1:
            print("相似詞前 30 排序")
            res = model.most_similar(q_list[0],topn = 30)
            for item in res:
                print(item[0]+","+str(item[1]))

        elif len(q_list) == 2:
            print("計算 Cosine 相似度")
            res = model.similarity(q_list[0],q_list[1])
            print(res)
        else:
            print("%s之於%s，如%s之於" % (q_list[0],q_list[2],q_list[1]))
            res = model.most_similar([q_list[0],q_list[1]], [q_list[2]], topn= 20)
            for item in res:
                print(item[0]+","+str(item[1]))
        print("----------------------------")
    except Exception as e:
        print(repr(e))

In [ ]:
# import json
# with open('../data/bigtable.json') as f:
#     BigAnalyzeTable=json.load(f)

In [ ]:
# pincutlist=[dien["contentcut"] for dien in BigAnalyzeTable]

In [ ]:
# from myapi import TextMining

In [ ]:
# from myapi import TextMining
# TM=TextMining.TextMining(pincutlist)
# wordCountlist=TM.getWordCountList()
# TF_IDFlist=TM.getTFIDF()